importとか諸々

In [12]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

データの読み込み

In [13]:
test = pd.read_csv('test.csv')
year1 = pd.read_csv('year1.csv')
year2 = pd.read_csv('year2.csv')
year3 = pd.read_csv('year2.csv')
year4 = pd.read_csv('year2.csv')

In [14]:
year1["year"] = 1
year2["year"] = 2
year3["year"] = 3
year4["year"] = 4
test["year"] = 5

データをまとめる

In [15]:
all_data = pd.concat([year1,year2,year3,year4])

In [16]:
all_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X58,X59,X60,X61,X62,X63,X64,id,TARGET,year
0,0.20055,0.37951,0.39641,2.0472,32.351,0.38825,0.24976,1.3305,1.1389,0.50494,...,0.87804,0.001924,8.416,5.1372,82.658,4.4158,7.4277,0,0,1
1,0.20912,0.49988,0.47225,1.9447,14.786,0,0.25834,0.99601,1.6996,0.49788,...,0.853,0,4.1486,3.2732,107.35,3.4,60.987,1,0,1
2,0.24866,0.69592,0.26713,1.5548,-1.1523,0,0.30906,0.43695,1.309,0.30408,...,0.76599,0.69484,4.9909,3.951,134.27,2.7185,5.2078,2,0,1
3,0.081483,0.30734,0.45879,2.4928,51.952,0.14988,0.092704,1.8661,1.0571,0.57353,...,0.94598,0,4.5746,3.6147,86.435,4.2228,5.5497,3,0,1
4,0.18732,0.61323,0.2296,1.4063,-7.3128,0.18732,0.18732,0.6307,1.1559,0.38677,...,0.86515,0.12444,6.3985,4.3158,127.21,2.8692,7.898,4,0,1


前処理

In [17]:
for i in all_data.iloc[:,0:-2]: 
  all_data[i] = pd.to_numeric(all_data[i],errors='coerce')
all_data = all_data.fillna(all_data.mean())

In [18]:
for i in test.iloc[:,0:-1]:
  test[i] = pd.to_numeric(test[i],errors='coerce')
test = test.fillna(test.mean())


In [ ]:
def remove_outliers(df, q=0.005):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(all_data)
all_data.iloc[:,0:-2] = all_data[mask]
mask = remove_outliers(test)
test.iloc[:,0:-1] = test[mask]

all_data = all_data.fillna(all_data.mean())
test = test.fillna(test.mean())

In [ ]:
# 取り除く列のリスト
not_use_columns = ['X37']
# 学習用データから列を削除する（PassengerIdは後ほど取り除く）
all_data.drop(not_use_columns, axis=1, inplace=True)
# テスト用データから列を削除する
test.drop(not_use_columns, axis=1, inplace=True)

Xとyの決定

In [ ]:
X = all_data.iloc[:, 0:-2]
test_X = test.iloc[:,0:-1]

In [ ]:
y = all_data.loc[:, 'TARGET']

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X)

データを分割

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

pca = PCA(n_components = 20)
X_pca = pca.fit_transform(all_data)

distortions = []

for k  in range(1,20):              
    km = KMeans(n_clusters=k,       
                init='k-means++',   
                n_init=10,          
                max_iter=300,       
                random_state=0)     
    km.fit(X_pca)           
    distortions.append(km.inertia_) 

# 結果を散布図に出力
plt.plot(range(1,20), distortions,marker='o') 
plt.xticks([i for i in range(1, 20)])         
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80, max_features='auto')
rf.fit(X_train, y_train)
print('Training done using Random Forest')
ranking = np.argsort(-rf.feature_importances_)
f, ax = plt.subplots(figsize=(11, 9))
sns.barplot(x=rf.feature_importances_[ranking], y=X_train.columns.values[ranking], orient='h')
ax.set_xlabel("feature importance")
plt.tight_layout()
plt.show()

In [25]:
cols = X_train.columns.values[ranking][0:3]
X_1 = all_data[cols]
test_1_X = test[cols]
y_1 = all_data.loc[:, 'TARGET']
from sklearn.model_selection import train_test_split
X_1_train, X_1_test, y_1_train, y_1_test= train_test_split(X_1, y_1,test_size=0.3,random_state=0)

グリッドサーチでlightgbmを実装

In [24]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import lightgbm as lgb

model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,
                  num_boost_round=1000, 
                  early_stopping_rounds=100
                 )

train_data = lgb.Dataset(X_1_train, label=y_1_train)
eval_data = lgb.Dataset(X_1_test, label=y_1_test)
params = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
}

skf = StratifiedKFold(n_splits=10,
                      shuffle=True,
                      random_state=0)
param_grid = {"n_estimators":[100,500,1000],
              "max_features": [1, 2, 3, 4, 5, 7, 10],
              "max_depth": [3,5,7,10,15,None], 
              "min_samples_leaf":  [1, 2, 4],
              "min_samples_split": [2, 5, 10]
             } 
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring = 'balanced_accuracy',
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)
acc = accuracy_score(y_test, grid_result.fit(X_1_train, y_1_train).predict(X_1_test))



ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
pred = Grid.predict(test_1_X)

スタッキング

In [ ]:
'''from sklearn.linear_model import LogisticRegression
models = {
    ("RandomForest", RandomForestClassifier()),
    ("GradientBoosting", GradientBoostingClassifier()),
    ("NeuralNetwork", MLPClassifier(max_iter = 10000,)),
    ("AdaBoost", AdaBoostClassifier())
    }
for name, model in models:
    acc = accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))
    print("Accuracy of", name, ":", acc)

from sklearn.ensemble import StackingClassifier
import multiprocessing
cores = multiprocessing.cpu_count()
stacking = StackingClassifier(estimators = models, final_estimator = MLPClassifier(), n_jobs = cores)
acc = accuracy_score(y_test, stacking.fit(X_train, y_train).predict(X_test))
print("Accuracy of Stacking :", acc)
'''

csvに出力

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'TARGET': pred})
submission.to_csv('submission.csv', index=False)